In [ ]:
!pip install transformers datasets accelerate -U peft evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
!pip install typing functools

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for functools
  Running setup.py clean for functools
Failed to build functools
ERROR: Could not build wheels for functools, which is required to install pyproject.toml-based projects


In [ ]:
import transformers
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer, pipeline, DataCollatorWithPadding, EvalPrediction
from datasets import load_dataset, load_metric
from evaluate import evaluator, load
from peft import PeftModelForQuestionAnswering, PromptEncoderConfig, PromptEncoder
from collections import Counter
from typing import Tuple, List, Dict, Any
import numpy as np
from functools import partial
import re
import string
import torch

# Load the SQuAD v2 dataset
dataset = load_dataset('squad_v2')
train_dataset = dataset['train']
eval_dataset = dataset['validation']

# Load the fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    encodings = tokenizer(
        examples['question'],
        examples['context'],
        truncation=True,
        padding='max_length',
        max_length=492,
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    for i, (context, answer) in enumerate(zip(examples['context'], examples['answers'])):
        start_position = None
        end_position = None

        if answer['answer_start']:
            start_idx = answer['answer_start'][0]
            end_idx = start_idx + len(answer['text'][0])

            offset_mapping = encodings['offset_mapping'][i]

            for j, (offset_start, offset_end) in enumerate(offset_mapping):
                if offset_start <= start_idx and offset_end > start_idx:
                    start_position = j
                if offset_start < end_idx and offset_end >= end_idx:
                    end_position = j
                    break

            if start_position is not None and end_position is not None:
                start_positions.append(start_position)
                end_positions.append(end_position)
            else:
                start_positions.append(0)
                end_positions.append(0)
        else:
            start_positions.append(0)
            end_positions.append(0)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

small_train_dataset = train_dataset.select(range(100))
small_eval_dataset = eval_dataset.select(range(20))

# Tokenize the dataset first
train_dataset = small_train_dataset.map(tokenize_function, batched=True)
eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

# P-tuning Configuration
prompt_encoder_config = PromptEncoderConfig(
    peft_type="P_TUNING",
    task_type="QUESTION_ANS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    encoder_reparameterization_type="MLP",
    encoder_hidden_size=768,
)

prompt_encoder = PromptEncoder(prompt_encoder_config)

# Create the model
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
peft_model = PeftModelForQuestionAnswering(model, prompt_encoder_config)
peft_model.print_trainable_parameters()

# Training arguments
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

# Define the evaluator
#task_evaluator = evaluator("question-answering")
#peft_model.task = "question-answering"

#def compute_metrics(pred):
 #       labels = pred.label_ids
  #      preds = pred.predictions
   #     preds = preds.argmax(-1)
    #    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
     #  return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}
# Normalization Function
def normalize_answer(s):
    """Lower text and remove punctuation, articles, and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

# F1 Score Calculation
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

# Exact Match Score Calculation
def exact_match_score(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def compute_metrics(eval_pred):
    predictions_tuple, labels_tuple = eval_pred.predictions, eval_pred.label_ids
    start_logits, end_logits = predictions_tuple
    start_positions, end_positions = labels_tuple

    f1 = 0.0
    exact_match = 0

    for i in range(len(start_positions)):
        start_pred = start_logits[i].argmax()
        end_pred = end_logits[i].argmax()

        pred_ans = tokenizer.decode(eval_dataset[i]['input_ids'][start_pred:end_pred + 1])
        true_ans = tokenizer.decode(eval_dataset[i]['input_ids'][start_positions[i]:end_positions[i] + 1])

        f1 += f1_score(pred_ans, true_ans)
        exact_match += exact_match_score(pred_ans, true_ans)

    return {'f1': f1/len(start_positions), 'exact_match': exact_match/len(start_positions)}


trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[task_evaluator],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,790,212 || all params: 110,681,860 || trainable%: 1.6174393888935368


AttributeError: ignored